## Init & Data Preprocessing

In [1]:
#when you extract dataSet, plz make cycle one by one, assuring validation set has all 3 labels

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "" # or 0 1 to use GPU
import tensorflow as tf
import pandas as pd
import  numpy as np
gpu_options = tf.GPUOptions(allow_growth=True)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
# sess = tf.Session(config=tf.ConfigProto(device_count={'gpu':0}))


In [2]:
train=pd.read_csv('trainall_data.csv', header=None) #pandas is easy to watch variables
label=pd.read_csv('trainall_label.csv', header=None)
trainn=train.loc[:, 0:15]
train1=trainn.values
label1=label.values

print trainn.describe()
print label.describe()


                0            1            2            3            4   \
count  1435.000000  1435.000000  1435.000000  1435.000000  1435.000000   
mean      0.947044     0.911287     0.800539     0.767264     0.978130   
std       0.102431     0.153221     0.257359     0.263953     0.051359   
min       0.502960     0.381320     0.346510     0.264100     0.861590   
25%       0.903890     0.823450     0.577355     0.546055     0.937640   
50%       0.932500     0.881400     0.750880     0.719080     0.963220   
75%       1.002750     1.016900     0.990880     0.965080     1.009800   
max       1.231600     1.275300     1.395700     1.369200     1.245000   

                5            6            7            8            9   \
count  1435.000000  1435.000000  1435.000000  1435.000000  1435.000000   
mean      0.881171     0.905825     0.882795     0.898563     0.803547   
std       0.115018     0.152825     0.178629     0.146430     0.278767   
min       0.718870     0.443260     0

## Model Part, haven't shuffled yet

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64,
                        #input_shape=(41000,),
                        activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(5, activation=tf.nn.sigmoid)
])

#todo test all optimizer

# model.compile(
#     optimizer='adam',
#     #optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
#                 #loss=tf.keras.losses.categorical_crossentropy,
#                 loss='sparse_categorical_crossentropy',
#                 #loss=tf.losses.softmax_cross_entropy,
#                 # pred=tf.nn.softmax(tf.add(tf.matmul(X1,W),b)) ,
#                 # cost =tf.reduce_mean(-tf.reduce_sum(label1*tf.log(pred),reduction_indices=1)),
#                 #loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [4]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
SIZE = (4,4)

## Try CNN in Keras

In [17]:
import numpy as np
import os
import tempfile

import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

import tensorflow as tf


if K.backend() != 'tensorflow':
    raise RuntimeError('This example can only run with the TensorFlow backend,'
                       ' because it requires the Datset API, which is not'
                       ' supported on other platforms.')


def cnn_layers(inputs):
    x = layers.Conv2D(32, (3, 3),
                      activation='relu', padding='valid')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(num_classes,
                               activation='sigmoid',
                               name='x_train_out')(x)
    return predictions

In [38]:
batch_size = 24
buffer_size = 100
steps_per_epoch = int(np.ceil(1435 / float(batch_size)))  # = 469
epochs = 5
num_classes = 5

In [41]:
from random import randint,shuffle
X2= []
label2= []
random_fetch=range(1435)
shuffle(random_fetch)

for i in range(1435):
    X2.append(np.reshape(train1[random_fetch[i]],(4,4)))
    label2.append(label1[random_fetch[i]])
        
X2=np.array(X2)
label2=np.array(label2)
trainn=X2

In [49]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape

(60000, 28, 28)

In [52]:
x_train=trainn
y_train=label2
# x_train = x_train.astype(np.float32) / 255
x_train=x_train.astype(np.float32)
x_train = np.expand_dims(x_train, -1)
# y_train = tf.one_hot(y_train, 1)
# y_train=label
# x_train.shape
y_train.shape

(1435, 5)

In [54]:
# Create the dataset and its associated one-shot iterator.
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size)
iterator = dataset.make_one_shot_iterator()

# Model creation using tensors from the get_next() graph node.
inputs, targets = iterator.get_next()
model_input = layers.Input(tensor=inputs)
model_output = cnn_layers(model_input)
train_model = keras.models.Model(inputs=model_input, outputs=model_output)

train_model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],
                    target_tensors=[targets])



ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_4/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,32], [3,3,32,64].

In [ ]:
model_input = Input(tensor=inputs)
model_output = cnn_layers(model_input)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',   
                 input_shape=(SIZE[0], SIZE[1],1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))

## Data Visualization

In [ ]:
train_a=np.reshape(X1[410:820,:,:],[410,16]) # acetone
label_a=label1[410:820]
train_vi=pd.DataFrame(train_a)
train_vi.describe()
print label_a
import matplotlib.pyplot as plt
train_vi.plot(kind='scatter',x=2,y=1)
plt.show()

## Creating 100 times dataSet, using randint
## Shuffling the dataSet

In [ ]:
tess=[]
for i in range(4):
    #tess[i]=9
    tess.append(i)

tess=np.array(tess)
tess.shape

In [ ]:
#test
import random
#tess=[]
# for i in range(10):
#     tess.append(shuffle(range(10)))  
li=range(10)    
print li
tess=random.shuffle(li)
print tess
print li
    

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.015),
    #optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
                #loss=tf.keras.losses.categorical_crossentropy,
                #loss='sparse_categorical_crossentropy',
                loss='binary_crossentropy',
                #loss=tf.losses.softmax_cross_entropy,
                # pred=tf.nn.softmax(tf.add(tf.matmul(X1,W),b)) ,
                # cost =tf.reduce_mean(-tf.reduce_sum(label1*tf.log(pred),reduction_indices=1)),
                #loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#X1=np.tile(X_train,(100,1,1))
#label1=np.tile(label,(100,1))

## if run 1110 samples, make sure the input labels are correct and not modified

# gpu_options = tf.GPUOptions(allow_growth=True)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

from random import randint,shuffle
X2= []
label2= []
random_fetch=range(1435)
shuffle(random_fetch)

for i in range(1435):
    X2.append(np.reshape(train1[random_fetch[i]],(4,4)))
    label2.append(label1[random_fetch[i]])
        
X2=np.array(X2)
label2=np.array(label2)

np.set_printoptions(threshold='none')

model.fit(X2, label2, batch_size=10, validation_split=0.1, epochs=5, )
model.evaluate(X2,label2)

# print label1[368:409]

 ### Appently, the data performance between meth part, ace part and eth part are not that good, i.e. if we use single raw data set extracting from a single experience. The performance would be great (shown as below) because of the data correlation and sigularity. 

### In a word, we can not determine if the data presented from our experiments can differ from each other that well. That is a very significant prerequsite. Assuming there are better network structure to explore, I think we can enhance this MLP with the following aspects.

## 1. More reasonable normalization method. Maybe instead of using the last time point, there are better solution.
## 2. Adding more layers as well as tuning parameters for better performance. 
## 3. When we do shuffling, try to use random search instead of grid seach.
## 4. Increasing the number of samples in trainingSet, however it has been experienced before and it was not that effective.

In [ ]:
train_a=X1[410:820,:,:] # acetone
label_a=label1[410:820]
# print train_a
# print label_a

In [ ]:
np.set_printoptions(threshold='none')
print train_a.shape
print label_a.shape
print label_a #acetone
print X.shape

In [ ]:
a=range(10)
shuffle(a)
print a
np.set_printoptions(threshold='none')
label1

In [ ]:
#shuffling the cycles for 410,acetone

from random import randint,shuffle
train_a=X1[410:820,:,:] # acetone
label_a=label1[410:820]

X2= []
label2= []
random_fetch=range(10)
shuffle(random_fetch)
train_ace=X1

for i in range(10):
    for k in range(41):
        X2.append(np.reshape(train_a[random_fetch[i]*41+k],(4,4)))
        label2.append(label_a[random_fetch[i]*41+k])   
    
X2=np.array(X2)
label2=np.array(label2)


print random_fetch
np.set_printoptions(threshold=3)
print X2
print label2

In [ ]:
tess=np.array([[1,2,3],[4,5,6]])
t=np.tile(tess,(2,1))
t

In [ ]:
np.set_printoptions(threshold='none')
print label_a
train_sub=train_a[0:369,:,:]
label_sub=label_a[0:369]  
print label_a[409]

In [ ]:
# gpu_options = tf.GPUOptions(allow_growth=True)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
from random import randint,shuffle

train_a=X2
label_a=label2

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64,
                        #input_shape=(41000,),
                        activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

for i in range(410):
    if label_a[i]==3:
        label_a[i]=1
    elif label_a[i]==4:
        label_a[i]=2

# tiling the data
train_sub=train_a[0:369,:,:]
train_lab=label_a[0:369]
valid_sub=train_a[369:410,:,:]
valid_lab=label_a[369:410]
m1=np.tile(train_sub,(100,1,1))
m2=np.tile(valid_sub,(100,1,1))
m3=np.tile(train_lab,(100,))
m4=np.tile(valid_lab,(100,))
label_input=np.concatenate([m3,m4])
train_input=np.concatenate([m1,m2])

train_input1=np.tile(train_a,(100,1,1))
label_input1=np.tile(label_a,(100,))


np.set_printoptions(threshold='none')

model.compile(
    optimizer='adam',
    #optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
                #loss=tf.keras.losses.categorical_crossentropy,
                loss='sparse_categorical_crossentropy',
                #loss=tf.losses.softmax_cross_entropy,
                # pred=tf.nn.softmax(tf.add(tf.matmul(X1,W),b)) ,
                # cost =tf.reduce_mean(-tf.reduce_sum(label1*tf.log(pred),reduction_indices=1)),
                #loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_input, label_input, validation_split=0.1, epochs=5)
model.evaluate(train_input,label_input)

# print label1[368:409]

In [ ]:
label_input.shape

In [ ]:
label_a.s

In [ ]:
tess=[]
tess=np.array(tess)

In [ ]:
tess

In [ ]:
sess.close()

In [ ]:
label1

In [ ]:
sess = tf.Session()
#print sess.run(model.fit)
sess.close()

In [ ]:
test=np.reshape(X1[399],(1,4,4))

In [ ]:
test.shape

In [ ]:
np.set_printoptions(threshold='none')
#label1
y

In [ ]:
label1[399]

In [ ]:
print(model.predict(test,batch_size=10,verbose=0))

In [ ]:
print(model.predict_classes(test,batch_size=10,verbose=0))